## Database

### Selecting databases

#### Four factors to consider when selecting databases
* structure of data
* query pattern
* amount or scale that you need to handle

#### Caching solution
* cosider to use a cache system (Redis) when you
  + call database very frequently
  + make a remote call to independent services with high latency
  
#### File storage solution
* consider to use Blob (binary large object) storage (s3) when you
  + need a data store for images, videos rahter than store data in database for queries
  + blob solutions are usually combined with a Content Delivery Network (CDN)

#### CDN
* A CDN is a network of servers around the world that delivers content in different geographical locations with reduced latency
* Generally, static files such as HTML/CSS/JS, photos, and videos are served from CDN
  + these files will not be changed frequently
  + Some CDNs such as Amazon's CloudFront support dynamic content. 
  + The site's DNS resolution will tell clients which server to contact
* types of CDN
  + push
    + CDN is updated whenever you upload files to your server
    + it is good that you have the control of the process
    + use this if you don't have too much static content, otherwise, it would be very expensive
  + pull
    + lazy load. Only download and update and cache a content from your server when uncached content is requested
    + slow if you are the first user requesting a content, such as an image, but faster for following users
* Advantages
  + Significantly improve performance because:
    + Users receive data from data centers that are closer to them
    + Your server doesn't have to serve contents that CDN fulfills  
* Disadvantages compared to databases: lack of the following functionalities
  + concurrent management for multiple users to access
  + grant different access rights to different users
  + scalability and availability when adding thousands of users
  + search content for different users in a short time
  
#### Elasticsearch with text search capability
* Consider to use Elesticsearch if you need to build a search functionality supporting fuzzy search
  + for example, search by movie, genre, actor, actress, director, etc
* Elasticsearch is not a database, which provide a guarantee that once stored, our data will not be lost unless we delete it
  + Elasticsearch offers no guarantee that our data will not be lost
  + we should never use search engines like Elasticsearch as our primary data source
* We can load the data to them from our primary database to reduce search latency and provide fuzzy and relevance-based text search

#### Time series database
* updated sequentially in an append-only format instead of random update
* have more bulk rads for a certain time ranges as opposed to random reads
* used for metric tacking systems such as grafana or cloud watch to watch how metric changes vs time
* calcualte how many people watched a video in the last 1 week, 10 days, 1 month, 1 year, and so on
* Examples of time series database is OpenTSDB and InfluxDB 

#### Data warehouse 
* a large database to dump all of the data available to perform analytics
* the systems are not used for regular transactions but offline reporting
* redshift, or s3/athena/EMR/spark

#### NoSQL
* RDBMS has fixed schema
* NoSQL databases can provide support for
  + unstructured, non-relational data
  + dynamic schema
  + distributed system with data stored on different servers/nodes
  + low latency with heavy data intensive workload
  + high throughput for IOPs
  + Advantages
    + simple design
      + you can store all employee data in one document instead of multiple tables reqiring join operations
    + horizontal scaling (TB to PB of data)
      + easy to scale up since all the data related to a specific employee are stored in the same document instead of many tables over nodes
      + spread data across multiple nodes and balance data and queries across nodes automatically
      + failed nodes are replaced automatically
    + high availability
      + provide redundant nodes with replicas and node replacement without down time
    + provide non-structured or semi-structured data support
    + cost 

#### Types of NoSQL databases
* key/value stores (DynamoDB and Redis)
  + Store key value pairs for high performance
  + Can store unique simple data or marshaled complex data structure as key (No index on key)
  + Allow easy partitioning and horizontal scaling
  + In-memory system (memcached and redis) can be very fast
  + Use cases: Session oriented apps (an unique ID is assigned to each session such as user id)
    + user profile info
    + recommendations to users
    + targeted promotions
    + discounts    
* Document stores (MongoDB, Google Cloud Firestore)
  + Designed to store and retrieve documents in formats like XML, JSON and BSON
  + Documents are composed of hierarchical tree data structure that can include maps, collections and scalar values
  + Schema free. Fields with the same name can have different types in different documents
  + Use cases: unstructured catalogue data, like JSON files or other complex strutured, hierarchical data
    + product catalogue
      + each product has thousands of attributes that can not be saved in RDBMS with a fixed schema
      + store each attribute in a single file for easy management and fast read access
    + content management apps such as blogs and video platforms
* Graph database (Neo4J, OrientDB, Infinite Graph)
  + Data stored as graphs with nodes and arc (relationships)
    + nodes represent entities and edges represent relationships
  + Designed for complex data models (many-many relationship and multiple foreign keys)
  + Optimized for data aggregations and graph-like queries
  + Use cases:
    + social apps  
* columnar databases (Cassandra, HBase, Amazon SimpleDB)
  + store data in columns instead of rows
  + enable access to all entries in the column quickly and efficiently
  + Use case: efficient for large amount of aggregation and data analysis queries.
    + dramatically decreases disk I/O and amount of data to load from disk
    + especially to get aggregation of one column over a specific range
    + can use columnar storage format such as parque
  


#### When to use SQL or NoSQL
* SQL (RDBMS such as mysql)
  + Data are structured and require transaction operations, 
  + Data are relational and need to maintain the relations between data by foreign key
  + Data has strict schema
  + There are needs for complex join operations  
* NoSQL
  + Data conain differnt data types and column names with complex query patterns, use document DB such as Mongo DB
  + Data contain fixed column names with few query patterns, but volume increases very fast, use columnar DB, such as cassandra and dynamo
* you can combine RDBMS with other nosql DB
  + you need to process transaction in Aamzon, but also need to store a large amount of transaction history data. 
    + use mysql to store data that have not complete transactions. After transactions are completed, move those data to cassandra
  + use Mongo DB to store all the transacton/purchase history data, and query the data for a specific aggregation such as different product id and customer id in transactions of surgar in the last three months, and use those id to query mysql and cassandra to get more specific data  

### Database Technology

#### Replica
* The process of storing the same data in multiple locations to improve data availability, accessibility, resilience and reliability
* We can have a copy of the main database. You need to sync the copies with main database
  + If main database fails or has network issues, the replica can take over
  + When main database is back, you need to sync it with the replica and swap roles back
  + This data redundancy provide an extra source for database operations
  + Replicas in multiple locations can increase availability when system in one location is down
  + You can split up traffic to multiple servers/replicas to improve throughput
* Types of replication
  + Master-slave/primary-secondary replication
    + slave instances are read-only that can be promoted to read/write nodes when master node is down
    + good for read heavy app
    + primary node responsible for wirte and synchronize data with other readers
    + eventually consistency
    + not appropriate for write heavy apps
  + multi-leader replication
    + multiple primary nodes that preocess writes and send them to all other primary and secondary nodes to replicate
    + useful for apps in which we can continue to work even if we are offline
    + needs to handle write conflicts among primary nodes
  + Master-master/peer-to-peer/leadless replication (DynamoDB)
    + both nodes are read/write nodes and share the workloads
* Benefits
  + Improve reliability and availability
    + If any server is down due to hardware or network issues, we can use other servers
  + Reduce latency
    + use servers closest to the requests to reduce network latency
* Cost of replica and technical issues
  + Always need to sync the main with replica databases whenever main is updated or has new records inserted
  + Whenever the sync of replicas to the main database has issues, we need to stop the transaction, to make sure the main and replicas are always consistent. Otherwise, the system will violate strong consistency
  + It will take time to sync all the databases
  + replicating huge volume of data increases cost for storage and processing 
  + If large amount of users are located in a few locations, then only servers in those locations will be called, and will be overloaded 
  + maintaining replicas for large data volumes requires network traffic and increases bandwidth consumption
  
#### Sharding
* A database architect to separate table rows into multiple tables, known as partitions
  + Each partition has the same schema and columns, and is run on a server
  + The data is then distributed across multiple servers, or shards
  + The purpose is to distribute load evenly, and avoid hotspots
* Vertical and Horizontal sharding
  + vertical sharding separates a table by columns into separate tables
  + horizontal sharding separates a table by rows into separate tables
* Advantages
  + improve peformance (latency and throughput)
    + less read/write traffic
    + high cache hits  
    + reduced index size for faster query
    + parallel writing operations 
  + less replication
  + Improve availability
    + Failures of several partitions or database servers will not fail the entire system. Other partitions/servers still work
    + If we provide replicas for shards, then we can use the back up for the failed shard 
* When to use sharding
  + when your system is overloaded and needs scale up
    + your database cannot handle the volume of requests
    + systems with millions of requests use sharding by default
    + not needed if you don't have a lot of workload
  + example:
    + if a single table is larger than a single database can accomodate, we will have to split it to multiple databases
      + assuming a single node can handle 50 GB data, we can estimate how many shards we need to have
* Hotspot problem
  + When one shard is accessed more often than other shards, This shard gets more traffic, and the benefit of sharding is lost
* Sharding techniques
  + Key based sharding/hash based sharding
    + Calculate which server to store a row based on a column using hashing, such as customer id, ip address or zip code
      + If the hash function gives the value of 1, we will distribute that row to shard 1 
      + By using consistent hashing, we can distribute data uniformly
    + Advantages:
      + Distribute data/load evenly to avoid hotspots
    + Disadvantages:
      + Difficult to dynamically add or remove servers
        + Reshard database is needed whenever a server is added or removed (consistent hashing does not help)
          + When a new server is added, need to rebalance the data into new server
          + If a server is down, other shards do not have the data for this server to cover it. Need to reshard data
      + Increased complexity in SQL queries with expensive join operations
  + Range based sharding
    + Divide data based on a range column (such as age)
      + Users with ages 0-18, 19-27 go to shard 1 and 2, respectively
    + Advantage
      + Easy to implement
    + Disadvantages
      + Hotspots results from unevenly distributed data on range
      + If most users are 20-25 years old, that shard is overloaded
  + Directory based sharding
    + Use a lookup table to map from shard key values to shards
      + Shard key is the column we use as shard key
      + easy to add new shards and re-distribute data between shards. All these operations are transparent to other services
      + good for database quickly increase
      + We always need to use the lookup table for shard read and write
    + Disadvantages
      + Need to consult lookup table for every read and write query
      + Database and network performance of lookup table impact app performance
      + Lookup table service is the single point of failure
        + If lookup service is done, the whole system is down and system availability is gone
  + Geo-based sharding
    + Shard data by user regions or locations
    + Disadvantages
      + Hotspots when most users are from only a few geo regions
  + Federation, or split a database by function
    + you can split a database to users and products
    + Advantages:
      + Less read/write traffic to each splitted database, and therefore, less replication lags
      + Write in parallel to improve throughput
      + Smaller database allows more data to fit into memory and increases cache hits
    + Disadvantages
      + Not effective if schema requires huge functions or tables
      + Need application logic to decide which tables to read and write
      + Introduce table joins 
      + more software, hardware and complexity

  + Hash-based sharding
    + apply a function to teh primary key and find/locate teh server taht sores the record. eg. hash(id) % number_of_nodes
    + use consistent hashing when adding or removing nodes
    + advantages
      + evenly distributed data
      + works well for key-value data
    + disadvantages
      + weak consistency
        + no foreign keys can be defined since data are scattered all around servers, and data are not organized by any common attributes as in classic RDBMS
        + hard to store relational data
          + for example, data from the same country/regions are not stored in the same table, making it impossible to use join queries and other RDBMS functions
* Where to put sharding logic
  + at application server 
    + requests are processed in two steps
      + request for data sent to server by a network request
      + server goes to shard directly and fetch data
    + problems
      + increases server workload
      + difficult to scale up or the system is big
  + at application server proxy 
    + server proxy is a reverse proxy to balance workload in internal network
    + server delegate the sharding tasks to reverse proxy
    + proxy finds the shard, read or write data and fetch data
  + another strategy we can consider is
    + each node/server has a copy of the mapping table
    + each table has a partition key column to link them

#### Partitioning
* Partition is to break large table to smaller tables, and sharding is to break a large database to smaller databases
* Advantages of partitioning
  + small files means faster queries
  + smaller index can fit in the memory
  + dropping partition is fast
  + comparing to sharding the entire database, partitioning only a table is more practically feasible
* Partition strategies
  + Partition by a list of values
    + We can partition an order table into three: placed orders, in-progress orders and completed orders
    + most of the records are in completed order table, which will not be queried often
    + advatages
      + records in in-progress and placed orders tables will be queried faster due to the smaller table size 
    + disadvantages
      + uneven data distribution
      + need to move data between tables
        + if a in-progress order is completed, need to move it to completed order table
  + Partition by range of dates
    + We can partition the order table by the range of the date the order is requested
    + advantages
      + easy to drop historical data by dropped tables rather than deleting rows
      + faster queries due to smaller table size
    + disadvantage
      + uneven distribution of data
   + partition by hashing of a key, such as primary key
     + calcuate the hash key based on the value of a key column
     + mode the hash key value by the number of tables we want
     + advantages
       + even distribution of the records
     + disadvantages
       + worse query performance if we want to query all the records in last month from order table since all the records are scattered in all partitioned tables
       + works only if the table is accessed by the key
       + changing number of partitions is difficult. need to re-partition
 
  + disadvantages of partitioning
    + complexiyt of maintenance (MySql doesn't support partitioning, have to do that in app)
    + scanning all partitions is expensive if the partition is not well designed
    + hard to maintain uniquness. The same IDs may exist in different tables
       

#### Index
* Index
  + index is built on columns. The purpose is for database to find the rows faster
* data structure of the index
  + Hashmap and dictionary are not good for range search
  + B tree + is used for range search
    + nodes in B tree have more than 2 child nodes
    + this reduces the height of the tree to speed up traverse with less jumps
    + don't need to rebanlance the tree all the time
    + only leaf nodes store values. All the root and internal nodes only store ranges
    + the leaf nodes are ordered and connected as a linked list to enable the fast traverse

### CAP Theorem
* C: Consistency
* A: Availability
* P: Partition tolerance

* When network partition occurs (e.g. network nodes become disconnected), system can't be both consistent and highly available
* In case of disconnection, node A and B may update or retrieve the same key with different values, there are two options
  + consistency and partition tolerance (CP)
    + block both update/read oerations until nodes are connected, you lose high availability
    + we always wati for a partitioned node to give consistent results. We may get time out error, or wait forever
    + choose this if atomic reads/writes is required
  + availability and partition tolerance (AP)
    + let both nodes update/read. System has inconsistent results
    + return the most recent version available that may be stale
    + allos for update to writes afer partition is resolved
    + choose this when
      + requirements have flexibility in when the data should be synchronized
      + system needs to continue to work even with external errors, such as shopping carts    
* Summary
  + CAP is more applicable to nosql system
  + Consistency: nodes may see the same data, but may not be able to write
  + Availability: all nodes are able to write, but may not see the same data

### Database Terminologies
* ACID
  + Atomic: each transaction is all or nothing
  + Consistency: any transaction will bring database from one valid state to another, meaning that all the constrains will be fulfilled
  + Isolation: Executing transactions concurrently has the same results as executing them serially
  + Durability: once a transaction is committed, it will remain so
* Denormalization
  + By denormalization, redundant copies of data are written into tables to avoid expensive table join operations. This improves read performance at the expense of some write performance
  + It applies for read heavy systems
  + Materialized view can help to handle the storage of redundant copy and make them consistent
  + Constraints help to keep redundant data consistent with an increase in the complexity of table design 
* BASE
  + BASE transaction model is based on CAP theory for distributed databases
  + Basically Available: the availability in terms of the CAP theorem is not insured. So a transaction could potentially not generate a response, but the system will try its best to do so.
  + Soft state: the state of the system could change over the time even without any input.
  + Eventually consistent: the system should become consistent over the time if it does not receive any input.
* Consistency patterns
  + Weak consistency
  + After a write, reads may or may not see it. A best effort approach is taken
  + Seen  in memcached
  + Real time use cases
    + VOIP, Video chart, and real time multiplayer games
    + miss what is spoken during connection loss in a phone call
  + Eventual consistency
    + After a write, reads will eventually see it (typically within milliseconds). Data is replicated asynchronously.
    + Highly available systems
    + DNS 
    + Email
  + Strong consistency
    + After a write, reads will see it. Data is replicated synchronously.
    + Use cases
      + File systems 
      + RDBMS
      + Works well in systems that need transactions